In [ ]:
# default_exp db110

# db110

> Classes and functions for the db110

In [ ]:
#hide
from nbdev.showdoc import *


In [ ]:
#export
import pandas as pd
import numpy as np
import regex
import string
#import ipdb
from pandas import DataFrame
import matplotlib.pyplot as plt
import time



## Functions

In [ ]:
#export
def readDB110():
    "Simple function to test"
    return 'DB110 Test'

In [ ]:
assert(readDB110()) == 'DB110 Test'

In [ ]:
#export
def getFieldNames(logfile):
    "Returns field names from a db110 datalog collected by GTS"
    nameList = []
    fields = []
    pattern = r'#\s\d*\s(?P<field>\w.*)' 
    fieldsRegex = regex.compile(pattern)
    #fieldsRegex = regex.compile(pattern)
    fh = open(logfile,'r')
    lines = fh.readlines()
    for line in lines:
        m = fieldsRegex.search(line)
        if m:
            field = m.group('field')
            field = field.replace(' ','_')
            field = field.replace(':','.')
            field = field.replace('.','_')
            nameList.append(field)
        else:
            pass
    fh.close()
    #debug_here()
    # try a list here in prep for pandas
    
    # this looks like a set up for my matlab routines
    #fieldNames = list(enumerate(nameList))
    
    # how about this 
    fieldNames = nameList
        
    # need a dictionary?
    #fieldNames = dict.fromkeys(nameList)
    
    return fieldNames

In [ ]:
names = getFieldNames('PitchT3.log')

In [ ]:
print(names)

['row_number', 'Absolute_sample_number', 'pitchGimbal_run_curMode', 'pitchGimbal_run_curPosition', 'pitchGimbal_run_setPosition', 'pitchGimbal_run_errPosition', 'pitchGimbal_run_curVelocity', 'pitchGimbal_run_velError', 'pitchGimbal_run_setVelocity', 'pitchGimbal_run_errVelocity', 'pitchGimbal_run_curCommand']


Graphing with fastbook module; Use to describe a test process or sequence

## Classes

## Test